In [25]:
import pandas as pd
input_path = 'ss_strategy_1min_7d_avg_adr_range_bars.check6'

In [27]:
df_sample = pd.read_csv(f'../tmp-data/{input_path}.csv')
df_sample.reset_index(drop=True, inplace=True)
df_sample['timestamp'] = pd.to_datetime(df_sample['timestamp'])
df_sample = df_sample.sort_values(by='timestamp')
df_sample['ordinal'] = df_sample.index + 1
df_sample.set_index('timestamp', inplace=True)
df_sample.sort_index(inplace=True)
df_sample

,volume,average_adr,Open,High,Low,Close,macd,macd_signal,macd_histogram,bb_upper,bb_lower,bb_distance,_tr,atr,rsi,signal,false_signal,ordinal
timestamp,,,,,,,,,,,,,,,,,,
2022-03-21 23:19:01,291.087,815.40,41096.18,41177.720,41014.64,41096.18,-22.365454,-23.314776,0.949322,41096.180060,41096.179940,0.000120,163.080,108.100000,0.000000,0,-1,1
2022-03-21 23:23:00,89.845,815.40,41014.64,41096.180,41008.10,41014.64,-27.861905,-24.224202,-3.637703,41134.457931,41044.312069,90.145862,88.080,102.742857,0.000000,0,-1,2
2022-03-21 23:24:00,428.359,815.40,41014.64,41096.180,40960.60,41014.64,-31.850727,-25.749507,-6.101220,41143.366328,41021.813672,121.552655,135.580,104.511429,0.000000,0,-1,3
2022-03-21 23:25:00,133.654,815.40,41014.64,41096.180,40995.90,41014.64,-34.612901,-27.522186,-7.090715,41146.410711,41005.179289,141.231423,100.280,103.744286,0.000000,-1,0,4
2022-03-21 23:26:00,99.772,815.40,41014.64,41096.180,40972.90,41014.64,-36.382548,-29.294258,-7.088290,41145.876649,40992.123351,153.753299,123.280,104.670000,0.000000,-1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-14 05:42:00,553.966,587.05,24317.79,24376.495,24290.00,24317.79,-29.444569,-33.274539,3.829969,24405.041954,24279.548046,125.493908,86.495,96.096786,40.288992,0,-1,100455
2023-03-14 05:43:00,317.460,587.05,24317.79,24376.495,24306.80,24317.79,-28.363818,-32.292395,3.928576,24385.560448,24284.308718,101.251730,69.695,94.561071,40.288992,0,-1,100456
2023-03-14 05:44:00,536.410,587.05,24317.79,24376.495,24290.00,24317.79,-27.193842,-31.272684,4.078842,24383.306271,24281.626229,101.680043,86.495,93.689643,40.288992,0,-1,100457


In [28]:
df_sample['bb_distance'].mean()

107.90736607405866

In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter
from bokeh.layouts import column
from bokeh.models import Range1d

# Create a ColumnDataSource object to hold the plot data
source = ColumnDataSource(data=dict(
    x=df_sample.index,
    y=df_sample['Close'],
    bb_upper=df_sample['bb_upper'],
    bb_lower=df_sample['bb_lower'],
    signal=df_sample['signal'],
    false_signal=df_sample['false_signal'],
    average_adr=df_sample['average_adr'],
    macd=df_sample['macd'],
    macd_signal=df_sample['macd_signal'],
    macd_histogram=df_sample['macd_histogram'],
    rsi=df_sample['rsi'],
    bb_dist=df_sample['bb_distance'],
    atr=df_sample['atr'],
    vol=df_sample['volume']
))

# Create views of the source using BooleanFilters
false_buy_filter = BooleanFilter([True if s==1 and fs==1 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
false_sell_filter = BooleanFilter([True if s==-1 and fs==1 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
false_buy_view = CDSView(source=source, filters=[false_buy_filter])
false_sell_view = CDSView(source=source, filters=[false_sell_filter])
buy_filter = BooleanFilter([True if s==1 and fs==0 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
sell_filter = BooleanFilter([True if s==-1 and fs==0 else False for s,fs in zip(source.data['signal'], source.data['false_signal'])])
buy_view = CDSView(source=source, filters=[buy_filter])
sell_view = CDSView(source=source, filters=[sell_filter])

# Create a Range1d object for the x-axis range shared by all three figures
x_range = Range1d(start=df_sample.index[0], end=df_sample.index[-1])

# Create a new figure object
p = figure(title='ss', x_axis_label='Date',
           y_axis_label='Price', width=1400, x_range=x_range)

# Plot the Close values as a line
p.line(x='x', y='y', source=source, legend_label='Close')

# Plot the bb_upper and bb_lower values as shaded areas
p.varea(x='x', y1='bb_upper', y2='bb_lower', source=source, alpha=0.2, legend_label='Bollinger Bands')

# Mark the long signals with green triangles and the short signals with red triangles
p.triangle(x='x', y='y', source=source, view=buy_view, color='green', size=10, legend_label='Buy Signal')
p.inverted_triangle(x='x', y='y', source=source, view=sell_view, color='red', size=10, legend_label='Sell Signal')

p.triangle(x='x', y='y', source=source, view=false_buy_view, color='black', size=10, legend_label='False Buy Signal')
p.inverted_triangle(x='x', y='y', source=source, view=false_sell_view, color='black', size=10, legend_label='False Sell Signal')


p_macd = figure(title='MACD', x_axis_label='Date', y_axis_label='Value', width=1400, x_range=x_range)
# Plot the average ADR as a line
p_macd.line(x='x', y='average_adr', source=source, color='orange', legend_label='Average ADR')

# Plot the MACD as two lines (macd and signal)
p_macd.line(x='x', y='macd', source=source, color='blue', legend_label='MACD')
p_macd.line(x='x', y='macd_signal', source=source, color='red', legend_label='Signal')

# Plot the MACD histogram as a bar chart
p_macd.vbar(x='x', top='macd_histogram', source=source, width=1.0, alpha=0.2, color='gray', legend_label='Histogram')

# Plot the RSI as a line in a separate figure
p_rsi = figure(title='RSI', x_axis_label='Date', y_axis_label='RSI Value', width=1400, x_range=x_range)
p_rsi.line(x='x', y='rsi', source=source, color='purple')

# Plot the bb distance as a line in a separate figure
p_bb_dist = figure(title='BB Dist', x_axis_label='Date', y_axis_label='Value', width=1400, x_range=x_range)
p_bb_dist.line(x='x', y='bb_dist', source=source, color='blue')

# Plot the ATR as a line in a separate figure
p_atr = figure(title='ATR', x_axis_label='Date', y_axis_label='Value', width=1400, x_range=x_range)
p_atr.line(x='x', y='atr', source=source, color='blue')

# Plot the Volume as a Vol in a separate figure
p_vol = figure(title='Volume', x_axis_label='Date', y_axis_label='Value', width=1400, x_range=x_range)
p_vol.line(x='x', y='vol', source=source, color='blue')

# Add a shaded area for target time periods
for start, end in [('2022-10-01 01:29:00', '2022-10-31 23:08:00'), ('2022-12-01 02:49:00', '2022-12-31 23:46:00')]:
    p.varea(x=[pd.Timestamp(start), pd.Timestamp(end)], y1=0, y2=5000, color='red', alpha=0.2)
    p_macd.varea(x=[pd.Timestamp(start), pd.Timestamp(end)], y1=0, y2=1000, color='red', alpha=0.2)
    p_rsi.varea(x=[pd.Timestamp(start), pd.Timestamp(end)], y1=0, y2=100, color='red', alpha=0.2)
    p_bb_dist.varea(x=[pd.Timestamp(start), pd.Timestamp(end)], y1=0, y2=6000, color='red', alpha=0.2)
    p_atr.varea(x=[pd.Timestamp(start), pd.Timestamp(end)], y1=0, y2=100, color='red', alpha=0.2)
    p_vol.varea(x=[pd.Timestamp(start), pd.Timestamp(end)], y1=0, y2=100000, color='red', alpha=0.2)

show(column(p ,p_atr, p_vol, p_bb_dist, p_macd, p_rsi))
# Save the plot as an HTML file
# output_file(f'../charts/{input_path}-stats.html')